In [ ]:
### Import required packages and load model

In [82]:
import cobra
import os
from os.path import join
import numpy as np
import matplotlib.pyplot as plt

from cobra import Reaction, Metabolite

model = cobra.io.read_sbml_model("420_model.xml")

The next two cells demonstrate useful tools to look at specific metabolites and reactions.

In [83]:
# To look at a specific metabolite (ID, name, chemical formula, reactions)
model.metabolites.get_by_id("glc__D_e")

Metabolite identifier,glc__D_e
Name,D-Glucose
Memory address,0x0113b03f98
Formula,C6H12O6
Compartment,e
In 2 reaction(s),"EX_glc__D_e, GLCpts"


In [84]:
# To look at a specific reaction (ID, Name, stoichiometry, bounds)
model.reactions.get_by_id("EX_glc__D_e")

Reaction identifier,EX_glc__D_e
Name,D-Glucose exchange
Memory address,0x0113228630
Stoichiometry,glc__D_e <=> D-Glucose <=>
GPR,
Lower bound,-10.0
Upper bound,1000.0


### Going through the cells in this notebook, compete the set of perturbations and answer the questions. You can find examples for similar problems at this site: http://cobrapy.readthedocs.io/en/latest/getting_started.html

#### Optimize the model. What is the biomass accumulation rate?

In [85]:
solution = model.optimize()
print("Biomass Accumulation Rate {:.3f}".format(solution.objective_value))
model.summary()

Biomass Accumulation Rate 0.917
IN FLUXES        OUT FLUXES    OBJECTIVES
---------------  ------------  ----------------------
o2_e      19.9   h2o_e  27.7   BIOMASS_Ecol...  0.917
glc__D_e  10     co2_e  21
nh4_e      5     h_e    18.4
pi_e       3.37


#### Change the lower bound of the glucose exchange rate to -20.0. What is the biomass accumulation rate now?

In [86]:
for r in model.reactions:
    name = r.name
    if "ex" in name.lower() and "glucose" in name.lower():
        print("Reaction: ID - \'{}\',  NAME - \'{}\', REACTION - {}".format(r.id,
                                                                            name,
                                                                            r.reaction))
        print("Current Lower Bound: {:.3f}".format(r.lower_bound))
        r.lower_bound = -20
        glu_ex_id = r.id
        print("New Lower Bound: {:.3f}".format(r.lower_bound))
        
solution = model.optimize()
print("Biomass Accumulation Rate {:.3f}".format(solution.objective_value))
model.summary()

Reaction: ID - 'EX_glc__D_e',  NAME - 'D-Glucose exchange', REACTION - glc__D_e <=> 
Current Lower Bound: -10.000
New Lower Bound: -20.000
Biomass Accumulation Rate 1.833
IN FLUXES        OUT FLUXES    OBJECTIVES
---------------  ------------  ---------------------
o2_e      39.9   h2o_e  55.3   BIOMASS_Ecol...  1.83
glc__D_e  20     co2_e  42
nh4_e     10     h_e    36.8
pi_e       6.74


#### Now change the lower bound of the glucose exchange reaction to 0.0. What is the biomass accumulation rate now? Why?

In [87]:
r = model.reactions.get_by_id(glu_ex_id)
r.lower_bound = 0
solution = model.optimize()
print("Biomass Accumulation Rate {:.3f}".format(solution.objective_value))
model.summary()

Biomass Accumulation Rate 0.000
IN FLUXES    OUT FLUXES    OBJECTIVES
-----------  ------------  ------------------
                           BIOMASS_Ecol...  0


#### What do exchange reactions represent? What do the upper and lower bounds of exchange reactions represent?

Exchange reactions represent the introduction or uptake of a metabolite into the system. The exchange reactions are formulated here as:
A <=>
Such that forward flux through the reaction leads to clearance of the metabolite, and negative flux through the reaction represents uptake of that metabolite.

#### Knockout the reaction GLCpts. What is the biomass accumulation rate now? Why?

In [88]:
# Your code goes here
model = cobra.io.read_sbml_model("420_model.xml")
r = model.reactions.get_by_id("GLCpts")
r.knock_out()
print("Reaction:\n\tID - \'{}\',\n\tNAME - \'{}\',\n\tREACTION - {}".format(r.id, 
                                                                    r.name,
                                                                    r.reaction))
solution = model.optimize()
print("Biomass Accumulation Rate {:.3f}".format(solution.objective_value))

Reaction:
	ID - 'GLCpts',
	NAME - 'D-glucose transport via PEP:Pyr PTS',
	REACTION - glc__D_e + pep_c --> g6p_c + pyr_c
Biomass Accumulation Rate 0.000


In [89]:
model = cobra.io.read_sbml_model("420_model.xml")
for r in model.reactions:
    rxn_str = r.reaction
    if "glc__D_e" in rxn_str:
        print("{} : {}".format(r.name, rxn_str))

D-Glucose exchange : glc__D_e <=> 
D-glucose transport via PEP:Pyr PTS : glc__D_e + pep_c --> g6p_c + pyr_c


Similar to removing glucose import, knocking out this reaction takes out the first step in glucose metabolism, effectively shutting down metabolism.